In [1]:
import random
import torch


words = open('names.txt', 'r').read().splitlines()
words = sorted(list(set(words))) # set cause uncontrollable randomness， sorted for reproducibility
max_len = max(len(w) for w in words)
random.seed(42)
random.shuffle(words)

chs = list(set(''.join(words + ['.']))) # add special token
chs = sorted(chs, reverse=False)
stoi = {ch: i for i, ch in enumerate(chs)}
itos = {i: ch for i, ch in enumerate(chs)}

# predict next token use previous tokens
vocab_size = len(chs)
block_size = max_len + 1
X, Y = [], []

for w in words:
    x = torch.zeros(max_len + 1, dtype=torch.long)
    y = torch.zeros(max_len + 1, dtype=torch.long)
    x[1:1+len(w)] = torch.tensor([stoi[ch] for ch in w])
    y[:len(w)] = torch.tensor([stoi[ch] for ch in w])
    y[len(w)+1:] = -1 # mask the loss at the inactive locations
    X.append(x)
    Y.append(y)

X = torch.stack(X)
Y = torch.stack(Y)
n1, n2  = int(0.8 * len(X)), int(0.9 * len(X))

X_train, X_val, X_test = X.tensor_split([n1, n2])
Y_train, Y_val, Y_test = Y.tensor_split([n1, n2])

show = 20
for x, y in zip(X_train[:show], Y_train[:show]):
    sx = ''.join(itos[i.item()] for i in x)
    sy = ''.join(itos[i.item()] for i in y if i.item() != -1)
    print(f'{sx} -> {sy}')

X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape


.aukai.......... -> aukai.
.ellanore....... -> ellanore.
.liem........... -> liem.
.aquarius....... -> aquarius.
.joangel........ -> joangel.
.wryn........... -> wryn.
.isabela........ -> isabela.
.astryd......... -> astryd.
.maleik......... -> maleik.
.emerick........ -> emerick.
.natasha........ -> natasha.
.kasandra....... -> kasandra.
.aevin.......... -> aevin.
.brason......... -> brason.
.naiara......... -> naiara.
.alanna......... -> alanna.
.raunak......... -> raunak.
.gohan.......... -> gohan.
.ivie........... -> ivie.
.alandis........ -> alandis.


(torch.Size([23595, 16]),
 torch.Size([2949, 16]),
 torch.Size([2950, 16]),
 torch.Size([23595, 16]),
 torch.Size([2949, 16]),
 torch.Size([2950, 16]))

# move to manual backprop

In [2]:
from dataclasses import dataclass
import torch.nn as nn
import torch.nn.functional as F
# manual backprop
from tiny_torch import *

@dataclass
class ModelConfig:
    block_size: int = block_size # length of the input sequences of integers
    vocab_size: int = vocab_size # the input integers are in range [0 .. vocab_size -1]
    # parameters below control the sizes of each model slightly differently
    n_layer: int = 4
    n_embd: int = 64
    n_embd2: int = 64
    n_head: int = 4
    dtype: torch.dtype = torch.float64

## trainer

In [3]:
def train_compare(model, model_t, lr=0.1, n_steps=100, eval_every=10, bs=32):
    loss_fn = CrossEntropyLoss3d()
    # train
    torch.manual_seed(42)
    for  step in range(n_steps):
        idx = torch.randint(0, X_train.shape[0], (bs,))
        x, y = X_train[idx], Y_train[idx]

        # --- torch ---
        # forward
        logits_t = model_t(x)
        loss_t = F.cross_entropy(logits_t.view(-1, logits_t.size(-1)), y.view(-1), ignore_index=-1)
        # backward
        loss_t.backward()
        # update
        for p_t in model_t.parameters():
            p_t.data -= lr * p_t.grad
            p_t.grad = None
        # --- manual ---
        # forward
        logits = model(x)
        loss = loss_fn(logits, y)
        # backward
        dlogits = loss_fn.backward()
        model.backward(dlogits)
        # update
        for p, g in zip(model.parameters(), model.grads()):
            p.data -= lr * g
        
        
        # eval
        if step % eval_every == 0:
            x, y = X_val, Y_val
            with torch.no_grad():
                logits_t = model_t(x)
                val_loss_t = F.cross_entropy(logits_t.view(-1, logits_t.size(-1)), y.view(-1), ignore_index=-1)
            logits = model(x)
            val_loss = loss_fn(logits, y)
            print(f'step {step:<8} || Train   || {loss.item():.15f} || Val   || {val_loss.item():.15f}')
            print(f'              || Train_t || {loss_t.item():.15f} || Val_t || {val_loss_t.item():.15f}')

## mlp

In [4]:
class MLP(Module):
    """
    takes the previous block_size tokens, encodes them with a lookup table,
    concatenates the vectors and predicts the next token with an MLP.

    Reference:
    Bengio et al. 2003 https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf
    """

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.vocab_size = config.vocab_size
        self.wte = Embedding(config.vocab_size + 1, config.n_embd, dtype=config.dtype) # token embeddings table
        # +1 in the line above for a special <BLANK> token that gets inserted if encoding a token
        # before the beginning of the input sequence
        self.mlp = Sequential([
            Linear(self.block_size * config.n_embd, config.n_embd2, dtype=config.dtype),
            Tanh(),
            Linear(config.n_embd2, self.vocab_size, dtype=config.dtype)
        ])
        self.mlp[-1].weight.data *= 0.1
        self.mlp[-1].bias.data *= 0.01
        n_params = sum(p.numel() for p in self.parameters())
        print("number of mlp parameters: %d" % (n_params,))
        self.config = config
    
    def parameters(self):
        return list(self.wte.parameters()) + list(self.mlp.parameters())
    
    def grads(self):
        return list(self.wte.grads()) + list(self.mlp.grads())

    def get_block_size(self):
        return self.block_size

    def __call__(self, idx, targets=None):

        # gather the word embeddings of the previous 3 words
        idx_buf = []
        embs = []
        for k in range(self.block_size):
            tok_emb = self.wte(idx) # token embeddings of shape (b, t, n_embd)
            idx_buf.append(idx.unsqueeze(-1))
            embs.append(tok_emb)
            idx = torch.roll(idx, 1, 1)
            idx[:, 0] = self.vocab_size # special <BLANK> token

        # concat all of the embeddings together and pass through an MLP
        x = torch.cat(embs, -1) # (b, t, n_embd * block_size)
        logits = self.mlp(x)

        # backward buffer
        self.idx_buf = torch.cat(idx_buf, -1) # (b, t, t)


        return logits
    
    def backward(self, grad):
        grad = self.mlp.backward(grad)
        # mlp backprop to wte
        b, t, _ = grad.shape # (b, t, n_embd * block_size)
        grad = grad.view(b * t * self.config.block_size, self.config.n_embd) # (b*t*block_size, n_embd)
        wte_weight = self.wte.weight
        wte_grad = torch.zeros_like(wte_weight)
        wte_grad.index_add_(dim=0, index=self.idx_buf.view(-1), source=grad)
        self.wte.weight_grad = wte_grad


In [5]:
class MLPtorch(nn.Module):
    """
    takes the previous block_size tokens, encodes them with a lookup table,
    concatenates the vectors and predicts the next token with an MLP.

    Reference:
    Bengio et al. 2003 https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf
    """

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.vocab_size = config.vocab_size
        self.wte = nn.Embedding(config.vocab_size + 1, config.n_embd) # token embeddings table
        # +1 in the line above for a special <BLANK> token that gets inserted if encoding a token
        # before the beginning of the input sequence
        self.mlp = nn.Sequential(
            nn.Linear(self.block_size * config.n_embd, config.n_embd2),
            nn.Tanh(),
            nn.Linear(config.n_embd2, self.vocab_size)
        )
        n_params = sum(p.numel() for p in self.parameters())
        print("number of mlp parameters: %d" % (n_params,))

    def get_block_size(self):
        return self.block_size

    def forward(self, idx, targets=None):

        # gather the word embeddings of the previous 3 words
        embs = []
        for k in range(self.block_size):
            tok_emb = self.wte(idx) # token embeddings of shape (b, t, n_embd)
            idx = torch.roll(idx, 1, 1)
            idx[:, 0] = self.vocab_size # special <BLANK> token
            embs.append(tok_emb)

        # concat all of the embeddings together and pass through an MLP
        x = torch.cat(embs, -1) # (b, t, n_embd * block_size)
        logits = self.mlp(x)

        return logits


In [6]:
config = ModelConfig(n_embd=8, n_embd2=24)
# models
torch.manual_seed(42)
model_mlp = MLP(config)
model_mlp_t = MLPtorch(config)
# copy weights
model_mlp_t.wte.weight.data = model_mlp.wte.weight.data.clone()
for i, (p_t, p) in enumerate(zip(model_mlp_t.mlp.parameters(), model_mlp.mlp.parameters())):
    if p.dim() == 2:
        p_t.data = p.data.clone().T # linear layer weight
    else:
        p_t.data = p.data.clone()


train_compare(model_mlp, model_mlp_t)

number of mlp parameters: 3995
number of mlp parameters: 3995
step 0        || Train   || 3.297640663881704 || Val   || 3.267504153916392
              || Train_t || 3.297640663881704 || Val_t || 3.267504153916391
step 10       || Train   || 3.024899204673497 || Val   || 3.022521506732622
              || Train_t || 3.024899204673497 || Val_t || 3.022521506732622
step 20       || Train   || 2.910541702548877 || Val   || 2.903174425168292
              || Train_t || 2.910541702548878 || Val_t || 2.903174425168292
step 30       || Train   || 2.862459630906429 || Val   || 2.846881946272189
              || Train_t || 2.862459630906429 || Val_t || 2.846881946272189
step 40       || Train   || 2.788301334625511 || Val   || 2.806272830481473
              || Train_t || 2.788301334625510 || Val_t || 2.806272830481473
step 50       || Train   || 2.816853557882264 || Val   || 2.773501814770002
              || Train_t || 2.816853557882264 || Val_t || 2.773501814770002
step 60       || Train   |

## rnn

In [7]:
class RNN(Module):

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.vocab_size = config.vocab_size
        self.n_embd = config.n_embd
        self.n_embd2 = config.n_embd2
        self.start = torch.zeros(1, config.n_embd2, dtype=config.dtype) # the starting hidden state
        self.wte = Embedding(config.vocab_size, config.n_embd, dtype=config.dtype) # token embeddings table
        self.Cw = Linear(config.n_embd + config.n_embd2, config.n_embd2, dtype=config.dtype) # rnn cell weight
        self.lm_head = Linear(config.n_embd2, self.vocab_size, dtype=config.dtype)
        num_params = sum(p.numel() for p in self.parameters())
        print("number of rnn parameters: %d" % (num_params,))
        # grads
        self.start_grad = None
    
    def parameters(self):
        return [self.start] + list(self.wte.parameters()) + list(self.Cw.parameters()) + list(self.lm_head.parameters())
    
    def grads(self):
        return [self.start_grad] + list(self.wte.grads()) + list(self.Cw.grads()) + list(self.lm_head.grads())

    def get_block_size(self):
        return self.block_size

    def __call__(self, x):
        b, t = x.size()
        emb = self.wte(x) # (b, t, n_embd)
        # sequentially iterate over the inputs and update the RNN state each tick
        hprev = self.start.expand((b, -1)) # expand out the batch dimension
        hiddens = []
        emb_cat_hprevs = []
        for i in range(t):
            xt = emb[:, i, :] # (b, n_embd)
            emb_i_cat_hprev = torch.cat([xt, hprev], dim=1)
            # --- rnn cell ---
            hi = self.Cw(emb_i_cat_hprev)
            hi = hi.tanh()
            # --------------
            hprev = hi
            hiddens.append(hi)
            emb_cat_hprevs.append(emb_i_cat_hprev)
        # decode the outputs
        hidden = torch.stack(hiddens, 1) # (b, t, n_embd2)
        logits = self.lm_head(hidden)
        # backward buffer
        self.hidden = hidden
        self.emb_cat_hprevs = emb_cat_hprevs
        return logits

    def backward(self, grad):
        hidden, emb_cat_hprevs = self.hidden, self.emb_cat_hprevs
        t = hidden.size(1)
        dhidden = self.lm_head.backward(grad)
        # logits grad to start, wte, Cw grad
        dembs = []
        dCw, dhprev = 0., 0.
        if self.Cw.bias is not None:
            dCw_bias = 0.
        for i in range(t-1, -1, -1):
            # hidden state grad, emb grad
            dhi = dhidden[:, i, :] + dhprev # grad from logits + grad from prev hidden state
            hi = hidden[:, i, :]
            dhi = (1 - hi**2) * dhi # grad of tanh
            demb_i_cat_dhi = dhi @ self.Cw.weight.T
            demb_i, dhprev = demb_i_cat_dhi.tensor_split([self.n_embd,], dim=1)
            dembs.append(demb_i)
            # cell weight grad
            emb_i_cat_hprev = emb_cat_hprevs[i]
            dCw += emb_i_cat_hprev.T @ dhi
            if self.Cw.bias is not None:
                dCw_bias += dhi.sum(dim=0)
        dstart = dhprev.sum(dim=0, keepdim=True)
        demb = torch.stack(dembs[::-1], 1)
        self.wte.backward(demb)
        self.start_grad = dstart
        self.Cw.weight_grad = dCw
        if self.Cw.bias is not None:
            self.Cw.bias_grad = dCw_bias

In [8]:
class RNNtorch(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.vocab_size = config.vocab_size
        self.n_embd = config.n_embd
        self.n_embd2 = config.n_embd2
        self.start = nn.Parameter(torch.zeros(1, config.n_embd2)) # the starting hidden state
        self.wte = nn.Embedding(config.vocab_size, config.n_embd) # token embeddings table
        self.Cw = nn.Linear(config.n_embd + config.n_embd2, config.n_embd2)
        self.lm_head = nn.Linear(config.n_embd2, self.vocab_size)
        num_params = sum(p.numel() for p in self.parameters())
        print("number of rnn parameters: %d" % (num_params,))

    def get_block_size(self):
        return self.block_size

    def forward(self, idx):
        b, t = idx.size()

        # embed all the integers up front and all at once for efficiency
        emb = self.wte(idx) # (b, t, n_embd)

        # sequentially iterate over the inputs and update the RNN state each tick
        hprev = self.start.T.expand((b, -1)) # expand out the batch dimension
        hiddens = []
        for i in range(t):
            xt = emb[:, i, :] # (b, n_embd)
            xh = torch.cat([xt, hprev], dim=1)
            ht = (self.Cw(xh)).tanh()
            hprev = ht
            hiddens.append(ht)

        # decode the outputs
        hidden = torch.stack(hiddens, 1) # (b, t, n_embd2)
        logits = self.lm_head(hidden)

        return logits

In [9]:
config = ModelConfig(n_embd=8, n_embd2=44)
# models
torch.manual_seed(42)
model_rnn = RNN(config)
model_rnn_t = RNNtorch(config)
# copy weights
model_rnn_t.wte.weight.data = model_rnn.wte.weight.data.clone()
for i, (p_t, p) in enumerate(zip(model_rnn_t.parameters(), model_rnn.parameters())):
    if p.dim() == 2 and i != 1: # skip the embedding layer
        p_t.data = p.data.clone().T # linear layer weight
    else:
        p_t.data = p.data.clone()

train_compare(model_rnn, model_rnn_t)

number of rnn parameters: 3807
number of rnn parameters: 3807
step 0        || Train   || 3.362432962612295 || Val   || 3.329043391170546
              || Train_t || 3.362432962612295 || Val_t || 3.329043391170547
step 10       || Train   || 3.171036656791207 || Val   || 3.114081619871966
              || Train_t || 3.171036656791206 || Val_t || 3.114081619871965
step 20       || Train   || 2.921689018592812 || Val   || 2.918088278837589
              || Train_t || 2.921689018592812 || Val_t || 2.918088278837589
step 30       || Train   || 2.807653806375523 || Val   || 2.797327611978270
              || Train_t || 2.807653806375522 || Val_t || 2.797327611978270
step 40       || Train   || 2.707653574650008 || Val   || 2.711660180532699
              || Train_t || 2.707653574650008 || Val_t || 2.711660180532699
step 50       || Train   || 2.712063772179941 || Val   || 2.660885418532094
              || Train_t || 2.712063772179941 || Val_t || 2.660885418532094
step 60       || Train   |

## gru

In [10]:
class GRU(Module):

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.vocab_size = config.vocab_size
        self.n_embd = config.n_embd
        self.n_embd2 = config.n_embd2
        self.start = torch.zeros(1, config.n_embd2, dtype=config.dtype) # the starting hidden state
        self.wte = Embedding(config.vocab_size, config.n_embd, dtype=config.dtype) # token embeddings table
        self.Cr = Linear(config.n_embd + config.n_embd2, config.n_embd2, dtype=config.dtype)
        self.Cbar = Linear(config.n_embd + config.n_embd2, config.n_embd2, dtype=config.dtype)
        self.Cz = Linear(config.n_embd + config.n_embd2, config.n_embd2, dtype=config.dtype)
        self.lm_head = Linear(config.n_embd2, self.vocab_size, dtype=config.dtype)
        num_params = sum(p.numel() for p in self.parameters())
        print("number of gru parameters: %d" % (num_params,))
        # grads
        self.start_grad = None
    
    def parameters(self):
        return [self.start] + list(self.wte.parameters()) + list(self.Cr.parameters()) + list(self.Cbar.parameters()) + list(self.Cz.parameters()) + list(self.lm_head.parameters())
    
    def grads(self):
        return [self.start_grad] + list(self.wte.grads()) + list(self.Cr.grads()) + list(self.Cbar.grads()) + list(self.Cz.grads()) + list(self.lm_head.grads())

    def get_block_size(self):
        return self.block_size

    def __call__(self, x):
        b, t = x.size()
        emb = self.wte(x) # (b, t, n_embd)
        # sequentially iterate over the inputs and update the RNN state each tick
        hprev = self.start.expand((b, -1)) # expand out the batch dimension
        hiddens = []
        emb_cat_hprevs, emb_cat_hprev_resets, hprevs, hbars, zs, rs = [], [], [], [], [], []
        for i in range(t):
            emb_i = emb[:, i, :] # (b, n_embd)
            # --- gru cell ---
            emb_i_cat_hprev = torch.cat([emb_i, hprev], dim=1)
            ri = self.Cr(emb_i_cat_hprev)
            ri = ri.sigmoid()
            hprev_reset = ri * hprev
            emb_i_cat_hprev_reset = torch.cat([emb_i, hprev_reset], dim=1)
            hbar = self.Cbar(emb_i_cat_hprev_reset)
            hbar = hbar.tanh()
            zi = self.Cz(emb_i_cat_hprev)
            zi = zi.sigmoid()
            hi = (1 - zi) * hprev + zi * hbar
            # backward buffer
            hiddens.append(hi)
            emb_cat_hprevs.append(emb_i_cat_hprev)
            emb_cat_hprev_resets.append(emb_i_cat_hprev_reset)
            hprevs.append(hprev)
            hbars.append(hbar)
            zs.append(zi)
            rs.append(ri)
            # update hprev
            hprev = hi
        # decode the outputs
        hidden = torch.stack(hiddens, 1) # (b, t, n_embd2)
        logits = self.lm_head(hidden)
        # backward buffer
        self.hidden = hidden
        self.emb_cat_hprevs = emb_cat_hprevs
        self.emb_cat_hprev_resets = emb_cat_hprev_resets
        self.hprevs = hprevs
        self.hbars = hbars
        self.zs = zs
        self.rs = rs
        return logits

    def backward(self, dlogits):
        hidden, emb_cat_hprevs, emb_cat_hprev_resets, hprevs, hbars, zs, rs = self.hidden, self.emb_cat_hprevs, self.emb_cat_hprev_resets, self.hprevs, self.hbars, self.zs, self.rs
        t = hidden.size(1)
        dhidden = self.lm_head.backward(dlogits)
        # logits grad to start, wte, Cw grad
        dembs = []
        dCr, dCbar, dCz, dhprev = 0., 0., 0., 0.
        if self.Cr.bias is not None:
            dCr_bias = 0.
        if self.Cbar.bias is not None:
            dCbar_bias = 0.
        if self.Cz.bias is not None:
            dCz_bias = 0.
        for i in range(t-1, -1, -1):
            # hidden state grad, emb grad
            dhi = dhidden[:, i, :] + dhprev # grad from logits + grad from prev hidden state
            dhbar = dhi * zs[i]
            dhprev = dhi * (1 - zs[i])
            dzi = dhi * (hbars[i] - hprevs[i])
            dzi = dzi * (1 - zs[i]) * zs[i]
            demb_i_cat_hprev = dzi @ self.Cz.weight.T
            dCz += emb_cat_hprevs[i].T @ dzi
            if self.Cz.bias is not None:
                dCz_bias += dzi.sum(dim=0)

            dhbar = dhbar * (1 - hbars[i]**2)
            demb_i_cat_hprev_reset = dhbar @ self.Cbar.weight.T
            dCbar += emb_cat_hprev_resets[i].T @ dhbar
            if self.Cbar.bias is not None:
                dCbar_bias += dhbar.sum(dim=0)

            demb_i, dhprev_reset = demb_i_cat_hprev_reset.tensor_split([self.n_embd,], dim=1)
            dri = dhprev_reset * hprevs[i]
            dhprev += dhprev_reset * rs[i]
            dri = dri * (1 - rs[i]) * rs[i]
            demb_i_cat_hprev += dri @ self.Cr.weight.T
            dCr += emb_cat_hprevs[i].T @ dri
            if self.Cr.bias is not None:
                dCr_bias += dri.sum(dim=0)
            demb_more, dhprev_more = demb_i_cat_hprev.tensor_split([self.n_embd,], dim=1)
            demb_i += demb_more
            dhprev += dhprev_more
            dembs.append(demb_i)
        dstart = dhprev.sum(dim=0, keepdim=True)
        demb = torch.stack(dembs[::-1], 1)
        self.wte.backward(demb)
        self.start_grad = dstart
        self.Cr.weight_grad = dCr
        if self.Cr.bias is not None:
            self.Cr.bias_grad = dCr_bias
        self.Cbar.weight_grad = dCbar
        if self.Cbar.bias is not None:
            self.Cbar.bias_grad = dCbar_bias
        self.Cz.weight_grad = dCz
        if self.Cz.bias is not None:
            self.Cz.bias_grad = dCz_bias

In [11]:
class GRUtorch(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size
        self.vocab_size = config.vocab_size
        self.start = nn.Parameter(torch.zeros(1, config.n_embd2, dtype=config.dtype)) # the starting hidden state
        self.wte = nn.Embedding(config.vocab_size, config.n_embd) # token embeddings table
        self.Cr = nn.Linear(config.n_embd + config.n_embd2, config.n_embd2)
        self.Cbar = nn.Linear(config.n_embd + config.n_embd2, config.n_embd2)
        self.Cz = nn.Linear(config.n_embd + config.n_embd2, config.n_embd2)
        self.lm_head = nn.Linear(config.n_embd2, self.vocab_size)
        num_params = sum(p.numel() for p in self.parameters())
        print("number of gru parameters: %d" % (num_params,))

    def get_block_size(self):
        return self.block_size

    def forward(self, idx):
        b, t = idx.size()
        emb = self.wte(idx) # (b, t, n_embd)
        hprev = self.start.T.expand((b, -1)) # expand out the batch dimension
        hiddens = []
        for i in range(t):
            xt = emb[:, i, :] # (b, n_embd)
            # --- gru cell ---
            xh = torch.cat([xt, hprev], dim=1)
            r = (self.Cr(xh)).sigmoid()
            hprev_reset = r * hprev
            xhr = torch.cat([xt, hprev_reset], dim=1)
            hbar = (self.Cbar(xhr)).tanh()
            z = (self.Cz(xh)).sigmoid()
            z = (self.Cz(xh)).sigmoid()
            ht = (1 - z) * hprev + z * hbar
            # --------------
            hprev = ht
            hiddens.append(ht)

        # decode the outputs
        hidden = torch.stack(hiddens, 1) # (b, t, n_embd2)
        logits = self.lm_head(hidden)

        return logits

In [12]:
config = ModelConfig(n_embd=8, n_embd2=27)
# models
torch.manual_seed(42)
model_gru = GRU(config)
model_gru_t = GRUtorch(config)
# copy weights
model_gru_t.wte.weight.data = model_gru.wte.weight.data.clone()
for i, (p_t, p) in enumerate(zip(model_gru_t.parameters(), model_gru.parameters())):
    if p.dim() == 2 and i != 1: # skip the embedding layer
        p_t.data = p.data.clone().T # linear layer weight
    else:
        p_t.data = p.data.clone()

train_compare(model_gru, model_gru_t)

number of gru parameters: 3915
number of gru parameters: 3915
step 0        || Train   || 3.361403145058750 || Val   || 3.352860787255950
              || Train_t || 3.361403145058750 || Val_t || 3.352860787255950
step 10       || Train   || 3.269461320582565 || Val   || 3.251121526050696
              || Train_t || 3.269461320582565 || Val_t || 3.251121526050696
step 20       || Train   || 3.178847946129618 || Val   || 3.166952995381364
              || Train_t || 3.178847946129618 || Val_t || 3.166952995381364
step 30       || Train   || 3.107527022687791 || Val   || 3.088581327911588
              || Train_t || 3.107527022687791 || Val_t || 3.088581327911588
step 40       || Train   || 3.039824486713404 || Val   || 3.015034751629193
              || Train_t || 3.039824486713405 || Val_t || 3.015034751629194
step 50       || Train   || 2.988462129260083 || Val   || 2.953215323540391
              || Train_t || 2.988462129260084 || Val_t || 2.953215323540391
step 60       || Train   |